# TaxiSimulation examples

- typical import

In [ ]:
using TaxiSimulation, RoutingNetworks, JLD

## Creating TaxiProblems

The `TaxiProblem` class represents problems we are trying to solve. Its most important features are:
- a `Network` object from the `RoutingNetworks` package, together with a `RoutingPaths` object from the same package. These types represent the static routing graph and directions that will be used by taxis. Another `RoutingPaths` object describes the costs. The hypothesis is that taxis will use shortest paths in time (and not in cost)
- a set of `Customers` (all customers to appear)
- a set of `Taxis` (all taxis on the map)

###  Synthetic problems

- We create here a typical small-sized routing problem on a synthetic city

In [ ]:
# the network
n = urbanNetwork(8, distance=800.)

# the travel-times and paths. Here with road-types maximal speeds 
# (see RoutingNetworks for more details)
routing = roadTypeRouting(n)

#usually,the costs are proportional to the times (but not necessarily)
# here the cost is $5 per hour of driving
costs = RoutingPaths(n, routing.times*5./3600.)

# We create the taxi problem
# - customerTime = nb of seconds to pickup or dropoff a customer
# - waitingCost = nb of $ per second of a taxi waiting. (here $1/hour)
pb = TaxiProblem(n,routing,costs,customerTime= 10., waitingCost = 1./3600.)

# The problem is still incomplete, we need to add customer and taxis
# we first add random customers, see `taxiproblem/randomproblem.jl` for more info
# 1h of customer pickups, 0.35 customers per node per hour, fare = $80/hour, 
# customers can wait up to 5min after pickup time, and call 30min before
addRandomCustomers!(pb, 3600., 0.35, hourFare=80., custWait=5.0*60, custCall= 30.0*60)

# 20 uniformly distributed identical random taxis, all available at beginning of simulation
addRandomTaxis!(pb, 20)

# save this problem locally for latter use
save("data/smallurb.jld", "pb", pb)
pb

### Real-world problems 

We present here the creation of a problem using Manhattan's taxi data

#### Constructing real-world networks
The first step is to create the desired routing network. This can be done with the package RoutingNetworks, from a polygon. In our case we will use the predefined MANHATTAN polygon.

In [ ]:
MANHATTAN_POLY = Tuple{Float32,Float32}[(-74.01369f0,40.69977f0), (-74.00597f0,40.702637f0), (-73.99944f0,40.70641f0), (-73.991714f0,40.708492f0), (-73.9761f0,40.71044f0), (-73.96923f0,40.72931f0), (-73.973526f0,40.736073f0), (-73.9615f0,40.75402f0), (-73.941765f0,40.774693f0), (-73.94348f0,40.78223f0), (-73.938156f0,40.78535f0), (-73.93593f0,40.79029f0), (-73.928894f0,40.79432f0), (-73.92872f0,40.803024f0), (-73.93318f0,40.80744f0), (-73.9349f0,40.833942f0), (-73.92134f0,40.85745f0), (-73.91893f0,40.858356f0), (-73.913956f0,40.863678f0), (-73.909706f0,40.872345f0), (-73.91829f0,40.875168f0), (-73.92648f0,40.879192f0), (-73.93344f0,40.87244f0), (-73.933525f0,40.86793f0), (-73.943436f0,40.853584f0), (-73.947945f0,40.85164f0), (-73.94713f0,40.84414f0), (-73.9552f0,40.828682f0), (-73.96091f0,40.8205f0), (-73.97734f0,40.79864f0), (-73.98957f0,40.78077f0), (-73.996994f0,40.770725f0), (-74.00352f0,40.761368f0), (-74.01064f0,40.75103f0), (-74.01532f0,40.719486f0), (-74.01764f0,40.719063f0), (-74.02047f0,40.704067f0)];

- get data (can take some time)

In [ ]:
manhattanNetwork = queryOsmPolygon(MANHATTAN_POLY)

- format the network to our need

In [ ]:
manhattanNetwork = roadTypeSubset(manhattanNetwork,1:6) # only keep main driving roads
manhattanNetwork = stronglyConnected(manhattanNetwork,1) # only keep the main connected component (hoping it's the one of node 1)
manhattanNetwork = intersections(manhattanNetwork) # simplify enormously the network, so that each node correspond to an intersection

- save the network to a julia JLD file for latter use

In [ ]:
saveTemplate(manhattanNetwork, "Manhattan")

- to load the network, just do:

In [ ]:
manhattanNetwork = loadTemplate("Manhattan")
# visualize(manhattanNetwork)

In [ ]:
visualize(manhattanNetwork)

#### Getting real taxi-data
For now, the package is able to parse and load CSVs of taxi trips from the NYC taxi and limousines commission. The CSVs can be found on their [website](http://www.nyc.gov/html/tlc/html/about/trip_record_data.shtml). The data is parsed and files can then be saved by date to simplify the reading.

In [ ]:
# Download data from website (will take some time, size = 1.8gb)
const DATA_URL_04_2016 = "https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2016-04.csv"
download(DATA_URL_04_2016, "data/taxidata042016.csv")

In [ ]:
# Parse data
realcustomers = loadManhattanCustomers(manhattanNetwork, "data/taxidata042016.csv")

# Save to files (one per day)
saveByDate(realcustomers, "data/manhattan")

# clean a little bit
rm("data/taxidata042016.csv")

- Finally, let's create the taxi problem

In [ ]:
# let's get some nice routing data created using another project
times = load("data/timingsestimates042016-0911-weekdays.jld", "times");
# if the previous file doe not exist, create virtual times instead:
# times = roadTypeRouting(manhattanNetwork).times

routing = RoutingPaths(manhattanNetwork, times)
costs = RoutingPaths(manhattanNetwork, routing.times*5./3600.)
pb = TaxiProblem(manhattanNetwork, routing, costs, customerTime=30., waitingCost = 1./3600.)

# Data from Wednesday 6th of April 2016 between 9 and 10:30
custData = load("data/manhattan-2016-04-06.jld", "customers")
startTime = DateTime(2016, 4, 6, 9, 0, 0)
endTime = startTime + Dates.Minute(90)

# adding 100% of the real demand
addDataCustomers!(pb, custData, startTime, endTime, 1.0)

# adding 3000 taxis with a distribution matching the customers pickups (to avoid "border effect")
addDistributedTaxis!(pb, 3000)

# updating customer behavior: they book between 0 and 20min earlier and wait up to 5min for a taxi.
pb = updateTcall(pb, 60*20.0,random=true)
pb = updateTmax(pb, 60*5.0)
save("data/manhattan.jld", "pb", pb)

pb